In [9]:
import pandas as pd
from matchms.importing import load_from_msp
from matchms.logging_functions import set_matchms_logger_level
import itertools

set_matchms_logger_level("ERROR")

In [10]:
predicted_spectra = list(load_from_msp("../data/filtered/simulated_matchms_filter_1%I_all_peaks.msp"))
predicted_spectra_metadata= pd.DataFrame.from_dict([x.metadata for x in predicted_spectra])
predicted_spectra_metadata.rename(columns={'compound_name': 'query'}, inplace=True)
predicted_spectra_names = predicted_spectra_metadata['query'].to_list()

In [11]:
experimental_spectra = list(load_from_msp("../data/experimental/RECETOX_GC-EI_MS_20201028.msp"))
experimental_spectra_metadata= pd.DataFrame.from_dict([x.metadata for x in experimental_spectra])
experimental_spectra_metadata.rename(columns={'compound_name': 'reference'}, inplace=True)
experimental_spectra_names = experimental_spectra_metadata['reference'].to_list()

In [12]:
all_combinations = set(itertools.product(predicted_spectra_names, experimental_spectra_names))

In [13]:
matches = pd.read_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5.tsv", sep="\t")
matches.set_index(['query', 'reference'], inplace=True)

In [14]:
existing_rows = set(matches.index.to_list())
rows_to_add = list(all_combinations - existing_rows)

In [15]:
all_zeros = pd.DataFrame(0, index=rows_to_add, columns=matches.columns)
merged = pd.concat([matches, all_zeros]).reset_index(names=['query', 'reference'])

In [16]:
merged.to_csv("../data/output_matching/matchms/matchms_tol_0.01_1%I_top5_with_0s.tsv", sep="\t", index=False)